In [1]:
import math,csv, random, collections
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from iced.normalization import ICE_normalization
from collections import Counter
import imp, sys, pickle
import utils, algorithms
%matplotlib inline


def get_var_matrix(points, data_file, sam_frac=1):
    Corr_matrix = algorithms.Measurement_matrix_variable(data_file, points, 1, 
                                                         columns=['chr', 'Pos1', 'Pos2'])

    Corr_matrix[np.diag_indices_from(Corr_matrix)] = 0
    Corr_matrix, bias = ICE_normalization(Corr_matrix, eps=1e-4, max_iter=1000, output_bias=True)
    
    return Corr_matrix

def get_const_matrix(length, data_file, sam_frac=1):
    Corr_matrix = algorithms.Measurement_matrix_equal(data_file, length, 1,
                                                              columns=['chr', 'Pos1', 'Pos2'])

    Corr_matrix[np.diag_indices_from(Corr_matrix)] = 0
    Corr_matrix, bias = ICE_normalization(Corr_matrix, eps=1e-4, max_iter=1000, output_bias=True)
    
    return Corr_matrix

def return_dist(M):
    #P
    P = M[utils.kth_diag_indices(M,1)]

    #Q
    A = np.tril_indices_from(M)
    arr1 = []
    arr2 = []
    for i in range(A[0].shape[0]):
        x = A[0][i]
        y = A[1][i]
        if abs(x-y)>1:
            arr1 += [x]
            arr2 += [y]
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    A_ = (arr1, arr2)
    Q = M[A_]
    
    return P, Q

def alpha_n(P,Q, bin_size):
    C1 = Counter(np.array(P/bin_size, dtype='int'))
    C2 = Counter(np.array(Q/bin_size, dtype='int'))
    support = list(set(C1.keys()).intersection( set(C2.keys())))
    sum1 = 0
    for key in support:
        sum1 += np.sqrt(C1[key]*C2[key])
    denom1 = np.sum(C1.values())+0.0
    denom2 = np.sum(C2.values())+0.0
    sum1 /= np.sqrt(denom1*denom2)
#     print  ">>", len(support), sum1, denom1, denom2
#     print C1, C2, "\n"
    return -2*np.log(sum1)/np.log(n)

def beta_n(P,Q,bin_size):
    C1 = Counter(np.array(P/bin_size, dtype='int'))
    C2 = Counter(np.array(Q/bin_size, dtype='int'))
    support = list(set(C1.keys()).intersection( set(C2.keys())))
    sum1 = 0
#     print "support length", len(support)
    for key in support:
        sum1 += np.power(C1[key]*C1[key]*C2[key], 1/3.)
    denom1 = np.sum(C1.values())+0.0
    denom2 = np.sum(C2.values())+0.0
    sum1 /= np.power(denom1*denom1*denom2, 1/3.)
#     print sum1, denom1, denom2
    return -1.5*np.log(sum1)/np.log(n)


In [264]:
it_limit = np.sqrt(4*np.log(n))
mg_limit = np.sqrt(6*np.log(n))
print it_limit
P = np.random.normal(loc=it_limit*.9, size=10000)
Q = np.random.normal(loc=0, size=10000)
plt.hist(P)
plt.hist(Q)
plt.show()
bin_size_array = np.array([0.1, 1, 3.14,10,25,50, 200])/20.
for bin_size in bin_size_array:
    alpha, beta = alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size) 
    print bin_size, alpha, beta, alpha/beta, alpha - beta

5.37114306959
0.005 1.09871646387 0.817523530616 1.3439569905 0.28119293325
0.05 0.919296008655 0.66725018103 1.37773811801 0.252045827625
0.157 0.890368447259 0.626377769716 1.42145601314 0.263990677543
0.5 0.848733163764 0.591060964132 1.43594859967 0.257672199632
1.25 0.785514067031 0.562299980324 1.39696620046 0.223214086706
2.5 0.518449223112 0.370323713459 1.39998926417 0.148125509653
10.0 -0.0 -0.0 nan 0.0


In [269]:
for chr in range(1,11):
    chromosome = str(chr)

    # Aeges

    # points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    # data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    # folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    # MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

    points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
    MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
    #Gaste
    
#     points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
#     MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))*10
        M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))*10
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
        M = get_const_matrix(100000, data_file)
        pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/2.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 0.281668159386 0.276114145334 64.421740904
0.25 0.278079332247 0.266600440272 64.421740904
0.5 0.277316212195 0.263932147475 64.421740904
0.75 0.276675917914 0.261998812025 64.421740904
2.5 0.275281464377 0.259117673176 64.421740904
5.0 0.271661934473 0.2557831864 64.421740904
12.5 0.227911095892 0.216866966517 64.421740904

chr 2
0.05 0.45775221606 0.451413167672 64.8079703692
0.25 0.454948847362 0.438911988856 64.8079703692
0.5 0.452472003444 0.430511274457 64.8079703692
0.75 0.452368404537 0.426867721722 64.8079703692
2.5 0.447929343791 0.416133601602 64.8079703692
5.0 0.4383073959 0.407403217935 64.8079703692
12.5 0.33002470652 0.313514632738 64.8079703692

chr 3
0.05 0.536348801488 0.533715214929 43.8046458183
0.25 0.535687401982 0.528114448976 43.8046458183
0.5 0.535415601642 0.525509023381 43.8046458183
0.75 0.535059627963 0.523581452951 43.8046458183
2.5 0.533481861032 0.517747210356 43.8046458183
5.0 0.53298219083 0.512340025089 43.8046458183
12.5 0.3966796008 0.37

In [271]:
for chr in range(1,11):
    chromosome = str(chr)

    # Aeges

    # points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    # data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    # folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    # MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

    points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
    MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
    #Gaste
    
#     points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
#     MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))*10
#         M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
#         M = get_const_matrix(100000, data_file)
#         pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/2.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 0.262237571064 0.244680687561 111.693382164
0.25 0.245794682608 0.222325118706 111.693382164
0.5 0.242366464426 0.215500668705 111.693382164
0.75 0.238957955743 0.211745236717 111.693382164
2.5 0.2321100093 0.202902738862 111.693382164
5.0 0.213983703199 0.18779489933 111.693382164
12.5 0.161207418394 0.144913986847 111.693382164

chr 2
0.05 0.207708483538 0.194182953211 189.081348755
0.25 0.1885699434 0.172259236411 189.081348755
0.5 0.186294572836 0.168083061535 189.081348755
0.75 0.18475009235 0.165146933244 189.081348755
2.5 0.18001899544 0.159157946513 189.081348755
5.0 0.168071654697 0.149038299292 189.081348755
12.5 0.133475253073 0.119845138927 189.081348755

chr 3
0.05 0.504433282849 0.468235421733 129.471824614
0.25 0.471109470113 0.423470954147 129.471824614
0.5 0.460411646866 0.41076463559 129.471824614
0.75 0.446568549534 0.398250412821 129.471824614
2.5 0.436038800064 0.381882523069 129.471824614
5.0 0.394835210421 0.348651342842 129.471824614
12.5 0.249865890

In [2]:
for chr in range(1,11):
    chromosome = str(chr)

    # Aeges

    # points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    # data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    # folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    # MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

    points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
    MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
#     Gaste
    
    points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
    MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))
        M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
        M = get_const_matrix(100000, data_file)
        pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/20.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 2.38408022268 1.63368530904 1055.20084492
0.25 1.9603878775 1.32602970301 1055.20084492
0.5 1.78019950513 1.18897667958 1055.20084492
0.75 1.72750342397 1.14896260627 1055.20084492
2.5 1.46400512038 0.945104766645 1055.20084492
5.0 1.3261891404 0.826548261993 1055.20084492
12.5 1.24064829313 0.764411646369 1055.20084492

chr 2
0.05 inf inf 699.196402974
0.25 2.282037901 1.51718775494 699.196402974
0.5 2.16038814938 1.42849535129 699.196402974
0.75 2.06524288951 1.35844547004 699.196402974
2.5 1.75240589427 1.12172088976 699.196402974
5.0 1.66315725127 1.0600833651 699.196402974
12.5 1.4901333734 0.928740456202 699.196402974

chr 3
0.05 inf inf 1420.47331152
0.25 2.38861649427 1.58878141014 1420.47331152
0.5 2.01863049772 1.33184465358 1420.47331152
0.75 1.93076941878 1.26695387374 1420.47331152
2.5 1.72095138761 1.13380100015 1420.47331152
5.0 1.6148921015 1.07312984729 1420.47331152
12.5 1.42238425432 0.948641147691 1420.47331152

chr 4
0.05 2.64179076943 1.76127744783 592

In [3]:
for chr in range(1,11):
    chromosome = str(chr)

    # Aeges

    # points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    # data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    # folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    # MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

    points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
    MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
    #Gaste
    
    points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
    MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))
#         M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
#         M = get_const_matrix(100000, data_file)
#         pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/20.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 0.32764829476 0.287993837099 1026.6272534
0.25 0.298316302318 0.249075748263 1026.6272534
0.5 0.287641269117 0.235395814768 1026.6272534
0.75 0.278760068895 0.228392907833 1026.6272534
2.5 0.262622942424 0.213981974784 1026.6272534
5.0 0.260953727764 0.211318270681 1026.6272534
12.5 0.258749719733 0.208126260818 1026.6272534

chr 2
0.05 0.435367471056 0.393788093515 1417.76556628
0.25 0.396512691663 0.339516474595 1417.76556628
0.5 0.381726879674 0.321538526539 1417.76556628
0.75 0.367386430441 0.306748506684 1417.76556628
2.5 0.338170665058 0.278564850234 1417.76556628
5.0 0.330457612453 0.269291038191 1417.76556628
12.5 0.323768528134 0.260975697882 1417.76556628

chr 3
0.05 0.357686102803 0.322972544139 974.890197981
0.25 0.331152779686 0.284932995801 974.890197981
0.5 0.319491719721 0.269208000007 974.890197981
0.75 0.307675350604 0.255726097254 974.890197981
2.5 0.280343557975 0.231249878081 974.890197981
5.0 0.271089524018 0.222948045016 974.890197981
12.5 0.264670505

In [4]:
for chr in range(1,11):
    chromosome = str(chr)

    # Aeges

    points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

#     points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
#     MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
    #Gaste
    
#     points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
#     MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))
        M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
        M = get_const_matrix(100000, data_file)
        pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/20.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 0.398972543326 0.338136160221 244.341717204
0.25 0.388575568249 0.317136924451 244.341717204
0.5 0.384725765546 0.312590001419 244.341717204
0.75 0.382074517233 0.310407192385 244.341717204
2.5 0.375518158522 0.305994709496 244.341717204
5.0 0.374860709747 0.304734464738 244.341717204
12.5 0.370296422296 0.301180586577 244.341717204
[Errno 2] No such file or directory: 'data/Aedes_aegypti/Measurement_matrix/MM_const_2.mtr'

chr 2
0.05 0.492946271241 0.410377480059 283.810841482
0.25 0.48283427844 0.386373188894 283.810841482
0.5 0.478866068994 0.380897493442 283.810841482
0.75 0.476146849944 0.378230046701 283.810841482
2.5 0.470044775498 0.37310175417 283.810841482
5.0 0.467926014814 0.371770909229 283.810841482
12.5 0.463525652519 0.368516523986 283.810841482
[Errno 2] No such file or directory: 'data/Aedes_aegypti/Measurement_matrix/MM_var_3.mtr'

chr 3
0.05 0.473586309907 0.397619369541 611.35971138
0.25 0.462500216193 0.372553217876 611.35971138
0.5 0.458026105953 0.36

IOError: [Errno 2] No such file or directory: 'data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr4'

In [5]:
for chr in range(1,4):
    chromosome = str(chr)

    # Aeges

    points = np.loadtxt('data/Aedes_aegypti/scaffold_positions/scaffold_positions_chr'+chromosome)
    data_file = 'data/Aedes_aegypti/HIC001/chromosomes/chr'+chromosome+'.data'
    folder = 'real_dataset_experiments/Aedes_aegypti/HIC001/error_rates/'
    MM_folder = 'data/Aedes_aegypti/Measurement_matrix/'


    ## Homo Sapiens

#     points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/HomoSapiens/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/HomoSapiens/error_rates/'
#     MM_folder = 'data/HomoSapiens/Measurement_matrix/'
    
    #Gaste
    
#     points = np.loadtxt('data/Gasterosteus/scaffold_positions/scaffold_positions_chr'+chromosome)
#     data_file = 'data/Gasterosteus/chromosomes/chr'+chromosome+'.data'
#     folder = 'real_dataset_experiments/Gasterosteus/error_rates/'
#     MM_folder = 'data/Gasterosteus/Measurement_matrix/'

    try:
        M = pickle.load(open( MM_folder+"MM_var_"+chromosome+".mtr", "rb" ))
#         M = pickle.load(open( MM_folder+"MM_const_"+chromosome+".mtr", "rb" ))
    except Exception as e:
        print e
        M = get_var_matrix(points, data_file)
        pickle.dump(  M, open( MM_folder+"MM_var_"+chromosome+".mtr", "wb" ) )
#         M = get_const_matrix(100000, data_file)
#         pickle.dump(  M, open( MM_folder+"MM_const_"+chromosome+".mtr", "wb" ) )
    n = M.shape[0]
    bin_size_array = np.array([1,5,10,15,50,100,250])/20.
    print "\nchr", chr
    P,Q = return_dist(M)
    for bin_size in bin_size_array:
        print bin_size, alpha_n(P,Q,bin_size), beta_n(P,Q,bin_size), M.max()


chr 1
0.05 0.425638276198 0.386157395611 2047.53434164
0.25 0.399855925193 0.339772912489 2047.53434164
0.5 0.387222559397 0.318105051823 2047.53434164
0.75 0.375238875464 0.303625111717 2047.53434164
2.5 0.3471798799 0.276598792353 2047.53434164
5.0 0.337481373379 0.267981053312 2047.53434164
12.5 0.332386731525 0.263156898654 2047.53434164

chr 2
0.05 0.482740010321 0.4382419905 4201.16571511
0.25 0.455428642411 0.402019154937 4201.16571511
0.5 0.434611574258 0.378078560091 4201.16571511
0.75 0.423559408536 0.362949045516 4201.16571511
2.5 0.394727492465 0.326922249285 4201.16571511
5.0 0.385643548516 0.315850056462 4201.16571511
12.5 0.371761298292 0.303273999738 4201.16571511

chr 3
0.05 0.658364506204 0.588645883687 5664.44637143
0.25 0.632803933327 0.555862656682 5664.44637143
0.5 0.607229330516 0.523350477223 5664.44637143
0.75 0.588692970035 0.501638260576 5664.44637143
2.5 0.554482447246 0.443684133889 5664.44637143
5.0 0.511512004024 0.40926936256 5664.44637143
12.5 0.477674

In [219]:
for chr in range(1,11):
    chromosome = str(chr)
    points = np.loadtxt('data/HomoSapiens/scaffold_positions/scaffold_positions_chr'+chromosome)
    lengths =  points[1:] - points[:-1]
    print chromosome, np.where(lengths<20000)[0].shape

1 (246,)
2 (308,)
3 (86,)
4 (94,)
5 (139,)
6 (145,)
7 (249,)
8 (84,)
9 (108,)
10 (110,)
